In [ ]:
%pip install tensorflow

In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Input, Dense, Bidirectional, LSTM

In [51]:
df = pd.read_excel('./data/3D_Data_Array_Processed.xlsx', header=[0, 1])

df.head()

Alcohol Intake                 Food Intake                 Water Intake  \
           Day 1   Day 2   Day 3       Day 1   Day 2   Day 3        Day 1   
0         0.3298  0.3293  0.3498      0.3136  0.3160  0.3180       0.3821   
1         0.3635  0.3518  0.3655      0.3560  0.3553  0.3538       0.5159   
2         0.1561  0.1554  0.1833      0.1487  0.1454  0.1489       0.5254   
3         0.6303  0.6085  0.6118      0.6438  0.6408  0.6374       0.6123   
4         0.2005  0.1987  0.2314      0.1780  0.1850  0.1890       0.3984   

                  Body Weight                  
    Day 2   Day 3       Day 1   Day 2   Day 3  
0  0.3820  0.4549      0.3125  0.3205  0.3760  
1  0.4694  0.5169      0.4164  0.4214  0.4737  
2  0.3311  0.3439      0.4339  0.3193  0.3527  
3  0.4698  0.3635      0.5215  0.4791  0.4484  
4  0.3429  0.4271      0.3109  0.3102  0.3579

## Initialize Model (Binary Classifier)
This model will ouptut a number between 0 and 1 to show the likelihood of alcholism in mice

### LSTM Model = Long Short Term Memory


### Other Model Specifications
- metrics for the model's accuracy

In [52]:
# Define model for simple BI-LSTM + DNN based binary classifier
def define_model():
    input1 = Input(shape=(3, 3))  # Update input shape to (3, 3) to match the 3D array dimensions
    lstm1 = Bidirectional(LSTM(units=32))(input1)
    dnn_hidden_layer1 = Dense(3, activation='relu')(lstm1)
    dnn_output = Dense(1, activation='sigmoid')(dnn_hidden_layer1)
    model = Model(inputs=[input1], outputs=[dnn_output])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

## Make 3-d numpy array

Example of Day 1 Data with mouse 1 (Alcohol, food, water, and body weight) with 3 time steps:


| Mouse Number | Time Step 1 | Time Step 2 | Time Step 3 | Features |
|--------------:|------------:|------------:|------------:|---------:|
| 1             | [0.3298, 0.3136, 0.3821, 0.3125	] | [0.3293, 0.316, 0.382, 0.3205] |  [0.3498, 0.318, 0.4549, 0.376]    | 4          |


Expected numpy array output: 

### Mouse #1:
Time Step 1: [0.3298, 0.3136, 0.3821, 0.3125]
ime Step 2: [0.3293, 0.316, 0.382, 0.3205]
Time Step 3: [0.3498, 0.318, 0.4549, 0.376]

In [54]:
alcohol_columns = [('Alcohol Intake', 'Day 1'), ('Alcohol Intake', 'Day 2'), ('Alcohol Intake', 'Day 3')]
food_columns = [('Food Intake', 'Day 1'), ('Food Intake', 'Day 2'), ('Food Intake', 'Day 3')]
water_columns = [('Water Intake', 'Day 1'), ('Water Intake', 'Day 2'), ('Water Intake', 'Day 3')]
body_weight_columns = [('Body Weight', 'Day 1'), ('Body Weight', 'Day 2'), ('Body Weight', 'Day 3')]

num_mice = len(df)
num_time_steps = 3
num_features = 4 

data_3d = np.zeros((num_mice, num_time_steps, num_features))

for i in range(num_mice):
    data_3d[i, :, 0] = df.loc[i, alcohol_columns].values
    data_3d[i, :, 1] = df.loc[i, food_columns].values
    data_3d[i, :, 2] = df.loc[i, water_columns].values
    data_3d[i, :, 3] = df.loc[i, body_weight_columns].values

for i, sample in enumerate(data_3d):
    print(f"\n### Mouse #{i + 1}:")
    for t, time_step in enumerate(sample):
        print(f"Time Step {t + 1}: {time_step}")


### Mouse #1:
Time Step 1: [0.3298 0.3136 0.3821 0.3125]
Time Step 2: [0.3293 0.316  0.382  0.3205]
Time Step 3: [0.3498 0.318  0.4549 0.376 ]

### Mouse #2:
Time Step 1: [0.3635 0.356  0.5159 0.4164]
Time Step 2: [0.3518 0.3553 0.4694 0.4214]
Time Step 3: [0.3655 0.3538 0.5169 0.4737]

### Mouse #3:
Time Step 1: [0.1561 0.1487 0.5254 0.4339]
Time Step 2: [0.1554 0.1454 0.3311 0.3193]
Time Step 3: [0.1833 0.1489 0.3439 0.3527]

### Mouse #4:
Time Step 1: [0.6303 0.6438 0.6123 0.5215]
Time Step 2: [0.6085 0.6408 0.4698 0.4791]
Time Step 3: [0.6118 0.6374 0.3635 0.4484]

### Mouse #5:
Time Step 1: [0.2005 0.178  0.3984 0.3109]
Time Step 2: [0.1987 0.185  0.3429 0.3102]
Time Step 3: [0.2314 0.189  0.4271 0.3579]

### Mouse #6:
Time Step 1: [0.2531 0.2186 0.0833 0.0833]
Time Step 2: [0.2669 0.2294 0.1025 0.0857]
Time Step 3: [0.3091 0.2436 0.2416 0.1561]

### Mouse #7:
Time Step 1: [0.7203 0.7309 0.3072 0.3249]
Time Step 2: [0.7171 0.7316 0.3502 0.3412]
Time Step 3: [0.7252 0.7275 0.4527 

In [56]:
print(data_3d)

[[[0.3298 0.3136 0.3821 0.3125]
  [0.3293 0.316  0.382  0.3205]
  [0.3498 0.318  0.4549 0.376 ]]

 [[0.3635 0.356  0.5159 0.4164]
  [0.3518 0.3553 0.4694 0.4214]
  [0.3655 0.3538 0.5169 0.4737]]

 [[0.1561 0.1487 0.5254 0.4339]
  [0.1554 0.1454 0.3311 0.3193]
  [0.1833 0.1489 0.3439 0.3527]]

 [[0.6303 0.6438 0.6123 0.5215]
  [0.6085 0.6408 0.4698 0.4791]
  [0.6118 0.6374 0.3635 0.4484]]

 [[0.2005 0.178  0.3984 0.3109]
  [0.1987 0.185  0.3429 0.3102]
  [0.2314 0.189  0.4271 0.3579]]

 [[0.2531 0.2186 0.0833 0.0833]
  [0.2669 0.2294 0.1025 0.0857]
  [0.3091 0.2436 0.2416 0.1561]]

 [[0.7203 0.7309 0.3072 0.3249]
  [0.7171 0.7316 0.3502 0.3412]
  [0.7252 0.7275 0.4527 0.3865]]

 [[0.8021 0.8375 0.6838 0.6195]
  [0.7895 0.8322 0.4185 0.4587]
  [0.7906 0.8211 0.5082 0.4944]]

 [[0.3449 0.3564 0.5099 0.4705]
  [0.3346 0.3527 0.3785 0.4123]
  [0.3516 0.3502 0.3825 0.4392]]

 [[0.3342 0.3128 0.4174 0.3179]
  [0.3281 0.3144 0.4302 0.3485]
  [0.3478 0.3169 0.4636 0.3841]]

 [[0.5215 0.5572 0.6

## Feed data into model